In [11]:
import numpy as np
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm
import cv2
import gym
import torch
from IPython.display import Image
from IPython.core.display import Image, display

import torchvision
from torchvision.utils import save_image
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable

from torch.optim import Adam
from torchsummary import summary

# CartPole-v1

In [12]:
env = gym.make('CartPole-v1')

states = []
actions = []
rewards = []
next_states = []
for i in tqdm(range(100)):

    obs = env.reset()
    done = False

    while not done:
        states.append(obs)
        
        action = env.action_space.sample()
        actions.append(action)
        
        obs, reward, done, _ = env.step(action)
        rewards.append(reward)
        next_states.append(obs)
        
env.close()

In [13]:
s_t = torch.from_numpy(np.array(states).astype(np.float32))
# a_t = torch.from_numpy(np.array(actions).astype(np.float32))  # for LunarLander-v0
a_t = torch.unsqueeze(torch.from_numpy(np.array(actions).astype(np.float32)), 1)
r_t = torch.unsqueeze(torch.from_numpy(np.array(rewards).astype(np.float32)), 1)
s_t1 = torch.from_numpy(np.array(next_states).astype(np.float32))

s_t.shape, a_t.shape, r_t.shape, s_t1.shape

(torch.Size([2318, 4]),
 torch.Size([2318, 1]),
 torch.Size([2318, 1]),
 torch.Size([2318, 4]))

# Models

* Forward: $f(s_t,a_t)=s_{t+1}$
* Backward: $f(s_{t+1}, a_t)=s_t$
* Inverse: $f(s_t,s_{t+1})=a_t$
* Reward: $f(s_{t+1})=r_{t+1}$, i.e. $f(s_t)=r_t$

In [14]:
class Reward(nn.Module):
    """Reward model predicts r_t1 from s_t1."""
    def __init__(self, s_dim):
        super(Reward, self).__init__()
        self.fc1 = nn.Linear(s_dim, 32)
        self.fc2 = nn.Linear(32, 16)
        self.fc3 = nn.Linear(16, 1)
    def forward(self, s_t1):
        x = F.relu(self.fc1(s_t))
        x = F.relu(self.fc2(x))
        r_hat_t1 = self.fc3(x)
        return r_hat_t1

In [15]:
class Forward(nn.Module):
    """Forward model predicts s_{t+1} from (s_t, a_t)."""
    def __init__(self, s_dim, a_dim):
        super(Forward, self).__init__()
        self.fc1 = nn.Linear(s_dim + a_dim, 32)
        self.fc2 = nn.Linear(32, 16)
        self.fc3 = nn.Linear(16, s_dim)
    def forward(self, s_t, a_t):
        x = torch.cat((s_t, a_t), dim=1)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        s_hat_t1 = self.fc3(x)
        return s_hat_t1

In [16]:
class Backward(nn.Module):
    """Backward model predicts s_t from (s_{t+1}, a_t)."""
    def __init__(self, s_dim, a_dim):
        super(Backward, self).__init__()
        self.fc1 = nn.Linear(s_dim + a_dim, 32)
        self.fc2 = nn.Linear(32, 16)
        self.fc3 = nn.Linear(16, s_dim)
    def forward(self, s_t1, a_t):
        x = torch.cat((s_t1, a_t), dim=1)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        s_hat_t = self.fc3(x)
        return s_hat_t

In [17]:
class Inverse(nn.Module):
    """Inverse model predicts a_t from (s_t, s_{t+1})."""
    def __init__(self, s_dim, a_dim):
        super(Inverse, self).__init__()
        self.fc1 = nn.Linear(2 * s_dim, 32)
        self.fc2 = nn.Linear(32, 16)
        self.fc3 = nn.Linear(16, a_dim)
    def forward(self, s_t, s_t1):
        x = torch.cat((s_t, s_t1), dim=1)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        a_hat_t = self.fc3(x)
        return a_hat_t

In [18]:
def f_loss(s_t1, s_hat_t1):
    return F.mse_loss(s_hat_t1, s_t1)

def b_loss(s_t, s_hat_t):
    return F.mse_loss(s_hat_t, s_t)

def i_loss(a_t, a_hat_t):
    return F.mse_loss(a_hat_t, a_t)

def r_loss(r_t, r_hat_t):
    return F.mse_loss(r_hat_t, r_t)

def fbi_loss(s_t, s_hat_t, a_t, a_hat_t, s_t1, s_hat_t1):
    return f_loss(s_t1, s_hat_t1) + b_loss(s_t, s_hat_t) + i_loss(a_t, a_hat_t)

In [19]:
f_f = Forward(s_dim=4, a_dim=1)
f_b = Backward(s_dim=4, a_dim=1)
f_i = Inverse(s_dim=4, a_dim=1)
f_r = Reward(s_dim=4)

f_opt = Adam(f_f.parameters())
b_opt = Adam(f_b.parameters())
i_opt = Adam(f_i.parameters())
r_opt = Adam(f_r.parameters())


fbi_losses = []
for i in range(100):
    f_opt.zero_grad()
    b_opt.zero_grad()
    i_opt.zero_grad()
    r_opt.zero_grad()
    
    s_hat_t1 = f_f(s_t, a_t)
    s_hat_t = f_b(s_t1, a_t)
    a_hat_t = f_i(s_t, s_t1)
    r_hat_t = f_r(s_t1)
    
    
    loss_f = f_loss(s_t1, s_hat_t1)
    loss_b = b_loss(s_t, s_hat_t)
    loss_i = i_loss(a_t, a_hat_t)
    loss_r = r_loss(r_t, r_hat_t)
    
    loss_f.backward()
    loss_b.backward()
    loss_i.backward()
    loss_r.backward()
    
    f_opt.step()
    b_opt.step()
    i_opt.step()
    r_opt.step()
    
    print(f"Epoch {i} - F: {loss_f:.2f}, B: {loss_b:.2f}, I: {loss_i:.2f}, Reward: {loss_r:.2f}")
    fbi_losses.append([loss_f, loss_b, loss_i, loss_r])

Epoch 0 - F: 0.25, B: 0.25, I: 0.57, Reward: 1.17
Epoch 1 - F: 0.25, B: 0.24, I: 0.56, Reward: 1.15
Epoch 2 - F: 0.25, B: 0.24, I: 0.55, Reward: 1.14
Epoch 3 - F: 0.25, B: 0.24, I: 0.54, Reward: 1.12
Epoch 4 - F: 0.24, B: 0.24, I: 0.54, Reward: 1.11
Epoch 5 - F: 0.24, B: 0.23, I: 0.53, Reward: 1.09
Epoch 6 - F: 0.24, B: 0.23, I: 0.52, Reward: 1.08
Epoch 7 - F: 0.24, B: 0.23, I: 0.51, Reward: 1.06
Epoch 8 - F: 0.23, B: 0.23, I: 0.51, Reward: 1.05
Epoch 9 - F: 0.23, B: 0.23, I: 0.50, Reward: 1.03
Epoch 10 - F: 0.23, B: 0.22, I: 0.49, Reward: 1.01
Epoch 11 - F: 0.23, B: 0.22, I: 0.49, Reward: 0.99
Epoch 12 - F: 0.22, B: 0.22, I: 0.48, Reward: 0.97
Epoch 13 - F: 0.22, B: 0.22, I: 0.47, Reward: 0.95
Epoch 14 - F: 0.22, B: 0.21, I: 0.46, Reward: 0.93
Epoch 15 - F: 0.22, B: 0.21, I: 0.46, Reward: 0.91
Epoch 16 - F: 0.21, B: 0.21, I: 0.45, Reward: 0.89
Epoch 17 - F: 0.21, B: 0.21, I: 0.44, Reward: 0.87
Epoch 18 - F: 0.21, B: 0.20, I: 0.43, Reward: 0.85
Epoch 19 - F: 0.21, B: 0.20, I: 0.43, Rew

# Model-Based Policy Optimization (MBPO)

* Based on the Dyna algorithm
1. Collect environment trajectories; add to $D_{env}$
2. Train model ensemble on environment data $D_{env}$
3. Perform k-step model rollouts branched from $D_{env}$; add to $D_{model}$
4. Update policy parameters on model data $D_{model}$

Source: [BAIR](https://bair.berkeley.edu/blog/2019/12/12/mbpo/)

In [10]:
import gym
import collections
import random

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

#Hyperparameters
learning_rate = 0.0005
gamma         = 0.98
buffer_limit  = 50000
batch_size    = 32

class ReplayBuffer():
    def __init__(self):
        self.buffer = collections.deque(maxlen=buffer_limit)
    
    def put(self, transition):
        self.buffer.append(transition)
    
    def sample(self, n):
        mini_batch = random.sample(self.buffer, n)
        s_lst, a_lst, r_lst, s_prime_lst, done_mask_lst = [], [], [], [], []
        
        for transition in mini_batch:
            s, a, r, s_prime, done_mask = transition
            s_lst.append(s)
            a_lst.append([a])
            r_lst.append([r])
            s_prime_lst.append(s_prime)
            done_mask_lst.append([done_mask])

        return torch.tensor(s_lst, dtype=torch.float), torch.tensor(a_lst), \
               torch.tensor(r_lst), torch.tensor(s_prime_lst, dtype=torch.float), \
               torch.tensor(done_mask_lst)
    
    def size(self):
        return len(self.buffer)

class Qnet(nn.Module):
    def __init__(self):
        super(Qnet, self).__init__()
        self.fc1 = nn.Linear(4, 128)
        self.fc2 = nn.Linear(128, 128)
        self.fc3 = nn.Linear(128, 2)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x
      
    def sample_action(self, obs, epsilon):
        out = self.forward(obs)
        coin = random.random()
        if coin < epsilon:
            return random.randint(0,1)
        else : 
            return out.argmax().item()
            
def train(q, q_target, memory, optimizer):
    for i in range(10):
        s,a,r,s_prime,done_mask = memory.sample(batch_size)

        q_out = q(s)
        q_a = q_out.gather(1,a)
        max_q_prime = q_target(s_prime).max(1)[0].unsqueeze(1)
        target = r + gamma * max_q_prime * done_mask
        loss = F.smooth_l1_loss(q_a, target)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

In [ ]:
def f_loss(s_t1, s_hat_t1):
    return F.mse_loss(s_hat_t1, s_t1)

In [66]:
class Policy(nn.Module):
    def __init__(self, s_dim, a_dim):
        super(Policy, self).__init__()
        self.fc1 = nn.Linear(s_dim, 32)
        self.fc2 = nn.Linear(32, 16)
        self.fc3 = nn.Linear(16, a_dim)
    def forward(self, s_t):
        x = F.relu(self.fc1(s_t))
        x = F.relu(self.fc2(x))
        a_t = F.relu(self.fc3(x))
        return a_t

In [ ]:
env = gym.make('CartPole-v0')

for n in range(500):
    

In [81]:
class Dream:
    def __init__(self, f_f, f_b=None, f_i=None, f_r=None, s_0=None):
        self.f_f = f_f
        self.f_b = f_b
        self.f_i = f_i
        self.f_r = f_r
        self.s_t = self.s_0 = s_0
    def reset(self):
        self.s_t = self.s_0
        return self.s_t
    def step(self, a_t):
        s_hat_t1 = self.f_f(self.s_t, a_t)
        r_hat_t = self.f_r(self.s_t)
        return s_hat_t1, r_hat_t

In [83]:
env = gym.make('LunarLander-v2')
s_0 = env.reset()

In [85]:
dream = Dream(f_f=f_f, f_b=f_b, f_i=f_i, f_r=f_r, s_0=s_0)

In [86]:
a_t

tensor([[2.],
        [3.],
        [0.],
        ...,
        [0.],
        [0.],
        [3.]])

In [ ]:
dream.step()